In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
def U():
    """
    This quantum function will simply contain H, T and CNOT gates.
    It will not return any value.
    """

    # Put your solution here #
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    
    qml.Hadamard(wires=0)
    qml.T(wires=0)
    qml.T(wires=1)
    qml.CNOT(wires=[1, 0])
    qml.T(wires=0)
    qml.T(wires=0)
    qml.T(wires=0)
    qml.CNOT(wires=[1, 0])
    qml.Hadamard(wires=0)
    
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    
    qml.CNOT(wires=[1, 0])

In [3]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit():
    """
    Main circuit given in the statement, here the operators you have defined in U will be embedded.
    """

    qml.CNOT(wires=[0, 1])

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    U()

    qml.CNOT(wires=[1, 0])

    U()

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    return qml.state()


# These functions are responsible for testing the solution.

def run(input: str) -> str:
    matrix = qml.matrix(circuit)().real
    #print(np.round(matrix, 2))

    with qml.tape.QuantumTape() as tape:
        U()

    names = [op.name for op in tape.operations]
    return json.dumps({"matrix": matrix.tolist(), "gates": names})

def check(user_output: str, expected_output: str) -> str:

    parsed_output = json.loads(user_output)
    matrix_user = np.array(parsed_output["matrix"])
    #print(matrix_user)
    gates = parsed_output["gates"]

    assert np.allclose(matrix_user, qml.matrix(qml.SWAP)(wires=[0, 1]))
    assert (
        len(set(gates)) == 3
        and "Hadamard" in gates
        and "CNOT" in gates
        and "T" in gates
    )


test_cases = [['No input', 'No output']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!


In [5]:
a=qml.matrix(qml.SWAP)(wires=[0, 1])
a

array([[1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1]])

In [35]:
b = qml.matrix(circuit)().real
b

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.34692639e-17],
       [ 0.00000000e+00, -1.34692639e-17,  1.00000000e+00,
         0.00000000e+00],
       [-1.34692639e-17,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00, -1.34692639e-17,
         0.00000000e+00]])

In [3]:
qml.prod(Sq)

AttributeError: Module pennylane has no attribute SqrtSwap

In [53]:
qml.SWAP(wires=[0, 1]).decomposition(), 

([CNOT(wires=[1, 0]), CNOT(wires=[0, 1]), CNOT(wires=[1, 0])],)

In [56]:
qml.matrix(qml.prod(qml.CNOT(wires=[0,1]), qml.CNOT(wires=[0, 1]), ))#, qml.T(wires=[0]), qml.T(wires=1))).real

ValueError: Require at least two operators to combine; got 1

In [155]:
qml.matrix(qml.prod(qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1])))

array([[1.+0.00000000e+00j, 0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 1.-6.66133815e-16j]])

In [48]:
qml.matrix(qml.prod(qml.Hadamard(0), qml.T(0)))

array([[ 0.70710678+0.j ,  0.5       +0.5j],
       [ 0.70710678+0.j , -0.5       -0.5j]])

In [57]:
qml.matrix(qml.CNOT(wires=[0, 1]))

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])

In [96]:
qml.CPhase(math.pi, wires=[0, 1]).decomposition()

[PhaseShift(1.5707963267948966, wires=[0]),
 CNOT(wires=[0, 1]),
 PhaseShift(-1.5707963267948966, wires=[1]),
 CNOT(wires=[0, 1]),
 PhaseShift(1.5707963267948966, wires=[1])]

In [125]:
op = qml.prod(qml.Hadamard(0), qml.CPhase(math.pi, wires=[0, 1]), qml.Hadamard(0))
op1 = qml.prod(qml.Hadamard(1), qml.CPhase(math.pi, wires=[1, 0]), qml.Hadamard(1))

In [129]:
np.round(qml.matrix(op).real, 3)

tensor([[ 1.,  0., -0.,  0.],
        [ 0., -0.,  0.,  1.],
        [-0.,  0.,  1.,  0.],
        [ 0.,  1.,  0., -0.]], requires_grad=True)

In [126]:
np.round(qml.matrix(qml.prod(op, op1, op)).real, 2)

tensor([[ 1., -0., -0., -0.],
        [-0., -0.,  1., -0.],
        [-0.,  1., -0., -0.],
        [-0., -0., -0.,  1.]], requires_grad=True)

In [144]:
np.round(qml.matrix(qml.PhaseShift(-math.pi/2, wires=1)).real, 3)

tensor([[1., 0.],
        [0., 0.]], requires_grad=True)

In [157]:
qml.PhaseShift(-math.pi/2, 1).decomposition()

[RZ(-1.5707963267948966, wires=[1])]

In [104]:
top0 = qml.prod(qml.T(wires=[0]), qml.T(wires=[0]), qml.T(wires=[0]), qml.T(wires=[0]))
top1 = qml.prod(qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]), qml.T(wires=[1]))
op11 = qml.prod(qml.Hadamard(0), top0, qml.CNOT(wires=[0, 1]), top1, qml.CNOT(wires=[0, 1]), top1, qml.Hadamard(0))
op22 = qml.prod(qml.Hadamard(1), top1, qml.CNOT(wires=[1, 0]), top0, qml.CNOT(wires=[1, 0]), top0, qml.Hadamard(1))

In [100]:
np.round(qml.matrix(qml.prod(op11, op22, op11)).real, 3)

tensor([[ 1., -0., -0.,  0.],
        [-0.,  1., -0., -0.],
        [-0., -0.,  1., -0.],
        [ 0., -0., -0.,  1.]], requires_grad=True)

In [107]:
np.round(qml.matrix(op11).real, 3)

tensor([[ 1.,  0., -0.,  0.],
        [ 0.,  1.,  0., -0.],
        [-0.,  0.,  1.,  0.],
        [ 0., -0.,  0.,  1.]], requires_grad=True)

In [139]:
np.round(qml.matrix(qml.prod(qml.PhaseShift(math.pi/2, 0), qml.CNOT(wires=[0, 1]), qml.PhaseShift(-math.pi/2, 1), qml.CNOT(wires=[0, 1]), qml.PhaseShift(math.pi/2, 1))).real, 3)

tensor([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0., -1.]], requires_grad=True)

In [137]:
qml.matrix(qml.prod(top0, qml.CNOT(wires=[0, 1]), top1,  qml.CNOT(wires=[0, 1]), top1)).real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [130]:
qml.matrix(qml.CPhase(math.pi, wires=[0, 1])).real

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.]])